<a href="https://colab.research.google.com/github/avijit7074/price_prediction_gurgaon/blob/master/model_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.preprocessing import OrdinalEncoder

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [3]:
train_df=pd.read_csv('/content/model_for_web.csv')
train_df.head()

,property_type,sector,price_in_cr,bedrooms,bathrooms,balconies,agePossession,built_up_area,study_room,servant_room,store_room,pooja_room,others,furnishing_type,luxury_category,floor_category
0,flat,sector 3,0.45,2.0,2.0,1,Relatively New,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,Low,Mid Floor
1,flat,sector 3,0.50,2.0,2.0,1,Old Property,722.0,0.0,0.0,0.0,0.0,0.0,1.0,Low,Low Floor
2,flat,sector 35,0.40,2.0,2.0,3,New Property,661.0,0.0,0.0,0.0,0.0,0.0,0.0,Low,High Floor
3,flat,sector 61,1.47,2.0,2.0,2,New Property,1333.0,1.0,0.0,0.0,0.0,0.0,0.0,Medium,Low Floor
4,flat,sector 92,0.70,2.0,2.0,3,Under Construction,1217.0,1.0,0.0,0.0,0.0,0.0,0.0,Low,Mid Floor


In [4]:
train_df.sample(5)

,property_type,sector,price_in_cr,bedrooms,bathrooms,balconies,agePossession,built_up_area,study_room,servant_room,store_room,pooja_room,others,furnishing_type,luxury_category,floor_category
0,flat,sector 3,0.45,2.0,2.0,1,Relatively New,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,Low,Mid Floor
2283,flat,sector 13,0.74,3.0,3.0,2,Relatively New,1361.0,0.0,0.0,0.0,0.0,0.0,0.0,Medium,Low Floor
2417,flat,sector 107,0.80,2.0,2.0,3+,New Property,1236.0,0.0,0.0,0.0,0.0,0.0,0.0,Medium,High Floor
702,flat,sector 99,1.10,2.0,2.0,3,New Property,1368.0,1.0,0.0,0.0,0.0,0.0,0.0,Medium,Mid Floor
2648,flat,sector 92,0.90,4.0,5.0,3+,Relatively New,1692.0,0.0,1.0,0.0,0.0,1.0,0.0,Medium,High Floor


In [5]:
train_df = train_df.drop(columns=['study_room','others', 'pooja_room'])

In [6]:
train_df.to_csv('model_for_web1.csv', index=False)

In [7]:
train_df = pd.read_csv('/content/model_for_web1.csv')
train_df.head()

,property_type,sector,price_in_cr,bedrooms,bathrooms,balconies,agePossession,built_up_area,servant_room,store_room,furnishing_type,luxury_category,floor_category
0,flat,sector 3,0.45,2.0,2.0,1,Relatively New,1000.0,0.0,0.0,0.0,Low,Mid Floor
1,flat,sector 3,0.50,2.0,2.0,1,Old Property,722.0,0.0,0.0,1.0,Low,Low Floor
2,flat,sector 35,0.40,2.0,2.0,3,New Property,661.0,0.0,0.0,0.0,Low,High Floor
3,flat,sector 61,1.47,2.0,2.0,2,New Property,1333.0,0.0,0.0,0.0,Medium,Low Floor
4,flat,sector 92,0.70,2.0,2.0,3,Under Construction,1217.0,0.0,0.0,0.0,Low,Mid Floor


In [8]:
train_df['floor_category'].unique()

array(['Mid Floor', 'Low Floor', 'High Floor'], dtype=object)

In [9]:
categorical_cols = train_df.select_dtypes(include=['object']).columns
categorical_cols

Index(['property_type', 'sector', 'balconies', 'agePossession',
       'luxury_category', 'floor_category'],
      dtype='object')

In [10]:
columns_to_encode = ['sector', 'balconies', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']
categorical_cols

Index(['property_type', 'sector', 'balconies', 'agePossession',
       'luxury_category', 'floor_category'],
      dtype='object')

In [11]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

# Define columns for ordinal encoding
#ordinal_cols = ['property_type', 'sector', 'balconies', 'agePossession','luxury_category', 'floor_category']

# Define columns for one-hot encoding
one_hot_cols = ['property_type','sector', 'balconies', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

# Create a ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedrooms', 'bathrooms', 'built_up_area', 'servant_room', 'store_room']),
        ('onehot', OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore'), one_hot_cols)
    ],
    remainder='passthrough'
)

# Create a scikit-learn pipeline that includes the preprocessor and a regressor
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor',SVR(C=1.0, epsilon=0.2, kernel = 'rbf'))
])

# Split your data into features (X) and the target (y)
X = train_df.drop('price_in_cr', axis=1)
y = train_df['price_in_cr']
y_transformed = np.log1p(y)

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)

# Fit and evaluate the pipeline
pipeline.fit(X_train, y_train)
score = pipeline.score(X_test, y_test)
print("Model Score:", score)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Model Score: 0.831889083478778


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [12]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/p

In [13]:
mean_value = round(scores.mean(), 2)
print('mean_value')

mean_value


In [14]:
scores

array([0.83455332, 0.83991536, 0.89037425, 0.85585413, 0.85070205,
       0.8692874 , 0.86837798, 0.85732003, 0.85510499, 0.83950839])

In [15]:
absolute_differences = np.abs(scores - mean_value)

# Calculate the MAE
mae = np.mean(absolute_differences)

print("Mean Absolute Error:", mae)

Mean Absolute Error: 0.013508137165426537


In [16]:
y_pred = pipeline.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [17]:
import pickle
pickle.dump(pipeline, open('pipe2.pkl','wb'))

In [18]:
train_df.loc[2023]

property_type                    flat
sector                     sector 113
price_in_cr                       2.0
bedrooms                          3.0
bathrooms                         3.0
balconies                           2
agePossession      Under Construction
built_up_area                  1507.0
servant_room                      0.0
store_room                        0.0
furnishing_type                   0.0
luxury_category                   Low
floor_category             High Floor
Name: 2023, dtype: object

In [19]:
z=X_test.loc[2023,:].to_frame().T
z

,property_type,sector,bedrooms,bathrooms,balconies,agePossession,built_up_area,servant_room,store_room,furnishing_type,luxury_category,floor_category
2023,flat,sector 113,3.0,3.0,2,Under Construction,1507.0,0.0,0.0,0.0,Low,High Floor


In [20]:
z=X.iloc[200, : ].to_frame().T
z

,property_type,sector,bedrooms,bathrooms,balconies,agePossession,built_up_area,servant_room,store_room,furnishing_type,luxury_category,floor_category
200,flat,sector 54,3.0,3.0,3+,Old Property,2558.0,0.0,0.0,1.0,Medium,Low Floor


In [21]:
pipeline.predict(z)

array([1.28083098])

In [22]:
np.expm1(1.12)

2.0648542032930024